In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import holidays
import itertools
import time

In [84]:
def import_ridership_data():
  # Import ridership data
  data_dir = '/content/drive/My Drive/MIE498 Thesis/0_Data'
  merged_bike_data_2019 = pd.read_csv("{}/ridership_2019_with_bike_stations_info_20200930.csv".format(data_dir), header=0)
  print(merged_bike_data_2019.shape)
  merged_bike_data_2019['End Day of Year'] = merged_bike_data_2019.apply(lambda row: datetime.strptime(row['End Time'], "%Y-%m-%d %H:%M:%S").timetuple().tm_yday, axis=1)
  print(merged_bike_data_2019.shape)

  merged_bike_data_2019['Start Time'] = merged_bike_data_2019.apply(lambda row: datetime.strptime(row['Start Time'], "%Y-%m-%d %H:%M:%S"), axis=1)
  merged_bike_data_2019['Start Day'] = merged_bike_data_2019.apply(lambda row: row['Start Time'].day, axis=1)
  merged_bike_data_2019['End Time'] = merged_bike_data_2019.apply(lambda row: datetime.strptime(row['End Time'], "%Y-%m-%d %H:%M:%S"), axis=1)
  merged_bike_data_2019['End Day'] = merged_bike_data_2019.apply(lambda row: row['End Time'].day, axis=1)

  return merged_bike_data_2019

In [85]:
def merge_clustering_data(nClusteringIterations, ridership_data, k2):
  merged_data = merged_bike_data_2019.copy()
  # Import cluster-station-assignment data
  df_data_w_clusters = pd.read_csv('/content/drive/My Drive/MIE498 Thesis/Share-Bike-Station-Clustering-and-Usage-Prediction/clustering_results/station_data_w_clusters_{}iterations_k2={}.csv'.format(nClusteringIterations, k2), index_col=0)
  merged_data = merged_data.merge(df_data_w_clusters[['station_id', 'cluster']], how='left', left_on='Start Station Id', right_on='station_id').drop('station_id', axis=1)
  merged_data.rename({'cluster': 'Start Cluster'}, axis=1, inplace=True)
  merged_data = merged_data.merge(df_data_w_clusters[['station_id', 'cluster']], how='left', left_on='End Station Id', right_on='station_id').drop('station_id', axis=1)
  merged_data.rename({'cluster': 'End Cluster'}, axis=1, inplace=True)

  return df_data_w_clusters, merged_data

In [86]:
def check_weekend(dayofweek):
    if dayofweek > 4:
      return 'weekend'
    else:
      return 'weekday'

In [87]:
def filter_checkout_checkin_data(merged_data):
# check-out and check-in data split
  checkout_data = merged_data[['Trip Id', 'Start Station Id', 'Start Time', 'Start Station Name', 'Start Year', 'Start Month', 'Start Hour',
        'Start Day of Week', 'Start Holiday', 'Start Day of Year', 'Start Week of Year', 'Start Lat', 'Start Lon', 'Start Cluster']]
  checkout_data['weekday/weekend'] = checkout_data.apply(lambda row: check_weekend(row['Start Day of Week']), axis=1)

  checkin_data = merged_data[['Trip Id', 'End Station Id', 'End Time', 'End Station Name', 'End Year', 'End Month',
        'End Hour', 'End Day of Week', 'End Holiday', 'End Lat', 'End Lon', 'End Day of Year', 'End Cluster']]
  checkin_data['weekday/weekend'] = checkin_data.apply(lambda row: check_weekend(row['End Day of Week']), axis=1)

  return checkout_data, checkin_data

## Helper Functions: Computing Objective Function Values (For Training Data Only):

In [88]:
def select_data(checkout_data, checkin_data):
    checkout_train = checkout_data.loc[(checkout_data['Start Month'] >= 1) & (checkout_data['Start Month'] <= 9)]
    checkin_train = checkin_data.loc[(checkin_data['End Month'] >= 1) & (checkin_data['End Month'] <= 9)]
    print(checkout_train.shape, checkin_train.shape)
    return checkout_train, checkin_train

In [89]:
def generate_transition_matrix_helper(station_id_list, mClusters, data_select, station_cluster_flag):
    if station_cluster_flag == 'cluster':
      from_list = list(np.arange(0, mClusters))
      df_transition_matrix = pd.DataFrame(data=np.zeros((mClusters, mClusters)), columns = from_list, index=from_list)
      column_name = 'Start Cluster'
    else:
      from_list = station_id_list
      nStations = len(station_id_list)
      df_transition_matrix = pd.DataFrame(data=np.zeros((nStations, mClusters)), columns = list(np.arange(0, mClusters)), index=from_list)
      column_name = 'Start Station Id'
    
    df_transition_matrix = df_transition_matrix.astype("int")

    for f in from_list:
      data_select[column_name] = data_select[column_name].astype("int")
      data_temp = data_select[data_select[column_name] == f]

      df_counts = pd.DataFrame(data_temp['End Cluster'].value_counts())
      
      # print(df_counts.head())
      to_cluster_list = list(df_counts.index)
      # print(to_cluster_list)
      for t in to_cluster_list:
        cnt = df_counts.loc[t]['End Cluster']
        df_transition_matrix.loc[f, t] = cnt
      
      df_transition_matrix = df_transition_matrix.div(df_transition_matrix.sum(axis=1), axis=0)

    df_transition_matrix.replace(np.nan, 0, inplace=True)
    return df_transition_matrix

In [90]:
def generate_transition_matrix_dict(station_id_list, mClusters, nTransitionMatrix, data, station_cluster_flag):
    transition_matrix_dict = {}
    if nTransitionMatrix == 1:
      return generate_transition_matrix_helper(station_id_list, mClusters, data, station_cluster_flag)
    
    elif nTransitionMatrix == 24:
      for hr in data['Start Hour'].unique():
        df_train_select = data[data['Start Hour'] == hr]
        transition_matrix = generate_transition_matrix_helper(station_id_list, mClusters, df_train_select, station_cluster_flag)
        transition_matrix_dict[hr] = transition_matrix
      return transition_matrix_dict
    
    elif nTransitionMatrix == 48:
      for hr in data['Start Hour'].unique():
        for wd in data['weekday/weekend'].unique():
          df_train_select = data[(data['Start Hour'] == hr) & (data['weekday/weekend'] == wd)]
          transition_matrix = generate_transition_matrix_helper(station_id_list, mClusters, df_train_select, station_cluster_flag)
          transition_matrix_dict[(hr, wd)] = transition_matrix
      return transition_matrix_dict

In [91]:
def fill_in_missing_combinations(mClusters, station_id_list, data, start_end_flag, station_cluster_flag):
    # Check missing combinations
    hours = np.arange(24)
    # dayofyear = data['{} Day of Year'.format(start_end_flag)].unique()
    weekday_weekend = ['weekday', 'weekend']
    if station_cluster_flag == 'Cluster':
      grouping_feature = np.arange(mClusters)
    else:
      grouping_feature = station_id_list

    combinations = itertools.product(hours, weekday_weekend, grouping_feature)
    features_list = ['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} {}'.format(start_end_flag, station_cluster_flag)]
    df_combinations = data[features_list].to_numpy().astype('str')
    df_combinations = df_combinations.tolist()
    comb_list = []
    for comb in combinations:
      comb = [str(comb[0]), comb[1], str(comb[2])]
      if comb not in df_combinations:
        # print(comb)
        comb_list.append(comb)
    print('number of missing combinations:', len(comb_list))

    # Fill in missing combinations
    for comb in comb_list:
      hour = int(comb[0])
      weekday_weekend = comb[1]
      grouping_feat = int(comb[2])
      df_select = data[(data['{} Hour'.format(start_end_flag)] == hour) & (data['weekday/weekend'] == weekday_weekend)]
      
      grouping_feature_list = data['{} {}'.format(start_end_flag, station_cluster_flag)].unique()
      for c in grouping_feature:
        if c not in grouping_feature_list:
              new_row = {'{} Hour'.format(start_end_flag) : hour, 'weekday/weekend' : weekday_weekend, 
                          '{} {}'.format(start_end_flag, station_cluster_flag) : grouping_feat,
                          'Trip Id': 0.000001}
              data = data.append(new_row, ignore_index = True)
    return data

In [92]:
def checkout_checkin_volume(mClusters, station_id_list, train_data, start_end_flag, station_cluster_flag):
  features_list = ['Trip Id', '{} Hour'.format(start_end_flag), 'weekday/weekend', '{} {}'.format(start_end_flag, station_cluster_flag)]
  grouping_features_list = ['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} {}'.format(start_end_flag, station_cluster_flag)]
  reset_levels_list = [0,1,2]

  train_data = train_data[features_list]
  train_data = train_data.groupby(by=grouping_features_list).count()
  train_data = train_data.reset_index(level=reset_levels_list)
  train_data['{} Hour'.format(start_end_flag)] = train_data['{} Hour'.format(start_end_flag)].astype("int64")

  train_data = fill_in_missing_combinations(mClusters, station_id_list, train_data, start_end_flag, station_cluster_flag)

  if start_end_flag == 'Start':
    name = 'Number of Checkouts ({})'.format(station_cluster_flag)
  else:
    name = 'Number of Checkins ({})'.format(station_cluster_flag)

  train_data = train_data.sort_values(by=grouping_features_list)
  train_data.rename({'Trip Id': name}, axis=1, inplace=True)
  train_data.reset_index(drop=True, inplace=True)

  print('Train Data Shape: ', train_data.shape)

  return train_data

In [93]:
def generate_checkout_checkin_dict_helper(station_id_list, mClusters, grouping_features, df_select, start_end_flag, station_cluster_flag):
  if start_end_flag == 'Start':
    checkout_checkin_flag = 'Checkouts'
  else:
    checkout_checkin_flag = 'Checkins'

  features_list = ['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} {}'.format(start_end_flag, station_cluster_flag), 'Number of {} ({})'.format(checkout_checkin_flag, station_cluster_flag)]
  reset_levels_list = list(np.arange(len(grouping_features)))
  # print(reset_levels_list)

  df_select = df_select[features_list]
  df_select = df_select.groupby(by=grouping_features).sum()
  if len(reset_levels_list) == 1:
    df_select = df_select.reset_index()
    df_select = df_select.drop(['{} Hour'.format(start_end_flag)], axis=1)
  else:
    df_select = df_select.reset_index(level=reset_levels_list)
    df_select['{} Hour'.format(start_end_flag)] = df_select['{} Hour'.format(start_end_flag)].astype("int64")

  if start_end_flag == 'Start':
    name = 'Number of Checkouts ({})'.format(station_cluster_flag)
  else:
    name = 'Number of Checkins ({})'.format(station_cluster_flag)

  df_select = df_select.sort_values(by=grouping_features)
  df_select.rename({'Trip Id': name}, axis=1, inplace=True)
  df_select.reset_index(drop=True, inplace=True)
  # print('Data Shape: ', df_select.shape)

  return df_select

In [94]:
def generate_checkout_checkin_dict(station_id_list, mClusters, grouping_features, data, start_end_flag, station_cluster_flag):
    return_dict = {}
    for c in range(mClusters):
      df_select = data[data['{} Cluster'.format(start_end_flag)] == c]
      checkout_checkin = generate_checkout_checkin_dict_helper(station_id_list, mClusters, grouping_features, df_select, start_end_flag, station_cluster_flag)
      return_dict[c] = checkout_checkin
    return return_dict

## Functions: Computing Objective Function Values (For Training Data Only):

In [95]:
def generate_station_cluster_dicts(nClusteringIterations, merged_bike_data_2019, k2):
  df_data_w_clusters, merged_data = merge_clustering_data(nClusteringIterations=nClusteringIterations, ridership_data=merged_bike_data_2019, k2=k2)
  merged_data_train = merged_data.loc[(merged_data['Start Month'] >= 1) & (merged_data['Start Month'] <= 9)]
  merged_data_train['weekday/weekend'] = merged_data_train.apply(lambda row: check_weekend(row['Start Day of Week']), axis=1)
  
  stationID_cluster = df_data_w_clusters[['station_id', 'cluster']]
  cluster_stationID_dict = {k: [] for k in stationID_cluster['cluster'].unique()}

  for idx, row in stationID_cluster.iterrows():
    station, cluster = row[0], row[1]
    cluster_stationID_dict[cluster].append(station)

  stationID_cluster_dict = dict(zip(stationID_cluster['station_id'], stationID_cluster['cluster']))
  
  clusters_list, station_id_list = list(merged_data['Start Cluster'].unique()), list(df_data_w_clusters['station_id'].unique())
  mClusters, nStations = len(clusters_list), len(station_id_list)

  return df_data_w_clusters, merged_data, merged_data_train, cluster_stationID_dict, stationID_cluster_dict, clusters_list, station_id_list, mClusters, nStations

In [96]:
def compute_metric1(nTransitionMatrix, merged_data_train, stationID_cluster_dict, clusters_list, station_id_list, mClusters, nStations):
  station_cluster_transitions = generate_transition_matrix_dict(station_id_list, mClusters, nTransitionMatrix, merged_data_train, station_cluster_flag='station')
  cluster_cluster_transitions = generate_transition_matrix_dict(station_id_list, mClusters, nTransitionMatrix, merged_data_train, station_cluster_flag='cluster')

  obj_val = 0
  if nTransitionMatrix == 1:
    for from_station_i in station_id_list:
      from_c = stationID_cluster_dict.get(from_station_i)
      for cluster_j in clusters_list:
        obj_val += abs(station_cluster_transitions.at[from_station_i, cluster_j] - cluster_cluster_transitions.at[from_c, cluster_j])
  else:
    if nTransitionMatrix == 24:
      iterating_list = range(nTransitionMatrix)
    elif nTransitionMatrix == 48:
      hours, weekday_weekend = np.arange(24), ['weekday', 'weekend']
      iterating_list = itertools.product(hours, weekday_weekend)
    
    for idx in iterating_list:
      for from_station_i in station_id_list:
        from_c = stationID_cluster_dict.get(from_station_i)
        for cluster_j in clusters_list:
          obj_val += abs(station_cluster_transitions[idx].at[from_station_i, cluster_j] - cluster_cluster_transitions[idx].at[from_c, cluster_j])
  obj_val = obj_val / (mClusters * nStations)

  return obj_val / nTransitionMatrix

In [97]:
def compute_metric2(nTransitionMatrix, cluster_stationID_dict, clusters_list, station_id_list, mClusters, nStations, df_counts, checkout_checkin_flag, start_end_flag):
  station_cluster_flag = 'Station Id'
  if nTransitionMatrix == 1:
    grouping_features = ['{} {}'.format(start_end_flag, station_cluster_flag)]
  elif nTransitionMatrix == 24:
    grouping_features = ['{} Hour'.format(start_end_flag), '{} {}'.format(start_end_flag, station_cluster_flag)]
  elif nTransitionMatrix ==48:
    grouping_features = ['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} {}'.format(start_end_flag, station_cluster_flag)]
  
  return_dict = generate_checkout_checkin_dict(station_id_list=station_id_list, mClusters=mClusters, grouping_features=grouping_features, data=df_counts, start_end_flag=start_end_flag, station_cluster_flag=station_cluster_flag)

  obj_val = 0
  for k in clusters_list:
    stations_list = cluster_stationID_dict.get(k)
    for i in stations_list:
      df_temp = return_dict[k]
      df_temp['Number of {} (Station Id)'.format(checkout_checkin_flag)] = return_dict[k]['Number of {} (Station Id)'.format(checkout_checkin_flag)].div(return_dict[k]['Number of {} (Station Id)'.format(checkout_checkin_flag)].sum(), axis=0)
      avg_checkouts = df_temp['Number of {} (Station Id)'.format(checkout_checkin_flag)].sum() / len(df_temp)
      for idx, row in df_temp.iterrows():
        n_checkouts = row[-1]  # of station i
        obj_val += abs(n_checkouts - avg_checkouts)
  obj_val = obj_val / (mClusters * nStations)
  
  return obj_val

## Computations

In [35]:
merged_bike_data_2019 = import_ridership_data()

(2438720, 26)
(2438720, 27)


In [100]:
df_metrics = pd.DataFrame(columns = ['n Clustering Iterations', 'k2', 'n Transition Matrix', 'Metric 1', 'Metric 2 (Checkout)', 'Metric 2 (Checkin)'])

In [101]:
n_clustering_iterations_list = [1, 5, 10, 15, 20]
k2_list = [8, 12, 16]
nTransitionMatrix_list = [1, 24, 48]

for nClusteringIterations in n_clustering_iterations_list:
  for k2 in k2_list:
    start_time = time.time()
    df_data_w_clusters, merged_data, merged_data_train, cluster_stationID_dict, stationID_cluster_dict, clusters_list, station_id_list, mClusters, nStations = generate_station_cluster_dicts(nClusteringIterations=nClusteringIterations, merged_bike_data_2019=merged_bike_data_2019, k2=k2)
    checkout_data, checkin_data = filter_checkout_checkin_data(merged_data)
    checkout_train, checkin_train = select_data(checkout_data, checkin_data)

    checkout_volume = checkout_checkin_volume(mClusters=mClusters, station_id_list=station_id_list, train_data=checkout_train, start_end_flag='Start', station_cluster_flag='Station Id')  # 'Station Id', 'Cluster'
    checkout_volume_with_cluster = checkout_volume.merge(df_data_w_clusters[['station_id', 'cluster']], how='left', left_on='{} Station Id'.format('Start'), right_on='station_id').drop('station_id', axis=1)
    checkout_volume_with_cluster.rename({'cluster': 'Start Cluster'}, axis=1, inplace=True)

    checkin_volume = checkout_checkin_volume(mClusters=mClusters, station_id_list=station_id_list, train_data=checkin_train, start_end_flag='End', station_cluster_flag='Station Id')  # 'Station Id', 'Cluster'
    checkin_volume_with_cluster = checkin_volume.merge(df_data_w_clusters[['station_id', 'cluster']], how='left', left_on='{} Station Id'.format('End'), right_on='station_id').drop('station_id', axis=1)
    checkin_volume_with_cluster.rename({'cluster': 'End Cluster'}, axis=1, inplace=True)

    for nTM in nTransitionMatrix_list:
      
      metric1 = compute_metric1(nTM, merged_data_train, stationID_cluster_dict, clusters_list, station_id_list, mClusters, nStations)
      metric2_checkout = compute_metric2(nTM, cluster_stationID_dict, clusters_list, station_id_list, mClusters, nStations, checkout_volume_with_cluster, 'Checkouts', 'Start')
      metric2_checkin = compute_metric2(nTM, cluster_stationID_dict, clusters_list, station_id_list, mClusters, nStations, checkin_volume_with_cluster, 'Checkins', 'End')
      df_temp = {'n Clustering Iterations' : nClusteringIterations, 'k2' : k2, 'n Transition Matrix' : nTM, 'Metric 1': metric1, 'Metric 2 (Checkout)': metric2_checkout, 'Metric 2 (Checkin)': metric2_checkin}
      df_metrics = df_metrics.append(df_temp, ignore_index=True)

    print(time.time() - start_time)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

(1970776, 15) (1970751, 14)
number of missing combinations: 22272
Train Data Shape:  (20707, 4)
number of missing combinations: 22272
Train Data Shape:  (20884, 4)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


638.9486005306244
(1970776, 15) (1970751, 14)
number of missing combinations: 22272
Train Data Shape:  (20707, 4)
number of missing combinations: 22272
Train Data Shape:  (20884, 4)
644.7488145828247
(1970776, 15) (1970751, 14)
number of missing combinations: 22272
Train Data Shape:  (20707, 4)
number of missing combinations: 22272
Train Data Shape:  (20884, 4)
645.5843987464905
(1970776, 15) (1970751, 14)
number of missing combinations: 22272
Train Data Shape:  (20707, 4)
number of missing combinations: 22272
Train Data Shape:  (20884, 4)
638.9947421550751
(1970776, 15) (1970751, 14)
number of missing combinations: 22272
Train Data Shape:  (20707, 4)
number of missing combinations: 22272
Train Data Shape:  (20884, 4)
644.6811270713806
(1970776, 15) (1970751, 14)
number of missing combinations: 22272
Train Data Shape:  (20707, 4)
number of missing combinations: 22272
Train Data Shape:  (20884, 4)
645.1675553321838
(1970776, 15) (1970751, 14)
number of missing combinations: 22272
Train 

In [102]:
df_metrics

,n Clustering Iterations,k2,n Transition Matrix,Metric 1,Metric 2 (Checkout),Metric 2 (Checkin)
0,1.0,8.0,1.0,0.009489,0.012553,0.012901
1,1.0,8.0,24.0,0.019558,0.021511,0.022515
2,1.0,8.0,48.0,0.022584,0.023987,0.024630
3,1.0,12.0,1.0,0.009070,0.011954,0.012226
4,1.0,12.0,24.0,0.018856,0.020725,0.021709
5,1.0,12.0,48.0,0.021806,0.023211,0.023829
6,1.0,16.0,1.0,0.008883,0.011633,0.012015
7,1.0,16.0,24.0,0.018534,0.020247,0.021215
8,1.0,16.0,48.0,0.021412,0.022617,0.023255
9,5.0,8.0,1.0,0.009628,0.012230,0.012575


In [103]:
savepath_metrics = '/content/drive/My Drive/MIE498 Thesis/Share-Bike-Station-Clustering-and-Usage-Prediction/objective_function_values.csv'
df_metrics.to_csv(savepath_metrics)